In [2]:
%load_ext autoreload
import cobra
import numpy as np
from importlib import reload
import mcs
#import traceback
#import warnings
import sys
# import ray


#ray.init(ignore_reinit_error=True)

#import mcs
#import contextlib
#import os,sys
#mcs_path = os.path.dirname(mcs.__file__)
#py_path = os.path.dirname(sys.executable)
#runtime_env = {"working_dir": ".", "py_modules": [mcs],"env_vars": {"PYTHON": py_path}}
#runtime_env = {"working_dir": ".","env_vars": {"PYTHON": py_path}}
#ray.shutdown()
#ray.init(ignore_reinit_error=True,runtime_env=runtime_env)
#with contextlib.redirect_stdout(None):


Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-03


In [18]:
%autoreload
# load network
network = cobra.io.read_sbml_model("weak_coupling.sbml")

# specify modules
modules = mcs.MCS_Module(network,"mcs_bilvl",module_sense="desired",constraints=["r_BM >= 0.5","r_P >= 1"],inner_objective="r_BM")

# specify MCS setup
ko_cost = {'r2': 1, 'r3': 1, 'r4': 1, 'r5': 1, 'r6': 1, 'r7': 1, 'r8': 1, 'r9': 1}
ki_cost = {'r1': 1}
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_smallest_mcs()

ValueError: When module type is "mcs_bilvl", an objective function must be provided.

In [14]:
%autoreload
# load network
network = cobra.io.read_sbml_model("SmallExample.sbml")

# specify modules
modules = [mcs.MCS_Module(network,"mcs_lin",module_sense="target",constraints=["R04 >= 1","2 R01 >= 0.5"])]
modules.append(mcs.MCS_Module(network,"mcs_lin",module_sense="desired",constraints="R03 >= 1"))

# specify MCS setup
ko_cost = {'R05': 2, 'R06': 3, 'R07': 4, 'R08': 4, 'R09': 4, 'R10': 4}
ki_cost = {'R02': 6}
maxSolutions = 50
maxCost = 50

# construct MCS MILP
mcsEnum = mcs.StrainDesigner(network,modules,ko_cost=ko_cost,ki_cost=ki_cost,max_cost=7,M=None)

# solve MILP
mcsEnum.compute_smallest_mcs()

{'R01': 0, 'R02': 0, 'R03': 0, 'R04': 0, 'R05': 0, 'R06': 0, 'R07': 1, 'R08': 0, 'R09': 0, 'R10': 0}
{'R01': 0, 'R02': 0, 'R03': 0, 'R04': 0, 'R05': 0, 'R06': 0, 'R07': 0, 'R08': 0, 'R09': 0, 'R10': 1}
{'R01': 0, 'R02': 0, 'R03': 0, 'R04': 0, 'R05': 0, 'R06': 1, 'R07': 0, 'R08': 1, 'R09': 0, 'R10': 0}


<3x10 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
ray.shutdown()
ray.is_initialized()

False

In [4]:
ray.shutdown()
ray.is_initialized()

False

In [16]:
import traceback
import warnings
import sys

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback
warnings.simplefilter("always")

In [11]:
#import ray
#ray.init()

@ray.remote
class timer(object):
    import time
    def __init__(self):
        self.starttime = self.time.monotonic_ns()
    def timepassed(self,x):
        self.time.sleep(0.1)
        return self.time.monotonic_ns()-self.starttime

parpool = ray.util.ActorPool([timer.remote() for _ in range(int(ray.available_resources()['CPU']))])

times = list(parpool.map(lambda a,x: a.timepassed.remote(x),range(20)))

{'__name__': 'mcs', '__doc__': None, '__package__': 'mcs', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, '__spec__': ModuleSpec(name='mcs', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000026A0BAB7EC8>, origin='c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', submodule_search_locations=['c:\\users\\phili\\documents\\python\\mcs\\mcs']), '__path__': ['c:\\users\\phili\\documents\\python\\mcs\\mcs'], '__file__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__init__.py', '__cached__': 'c:\\users\\phili\\documents\\python\\mcs\\mcs\\__pycache__\\__init__.cpython-37.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImpor

In [8]:
import time
starttime = time.monotonic_ns()
time.sleep(0.5)
print(time.monotonic_ns()-starttime)

516000000
